In [5]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import ast
import folium
from folium import plugins
import polyline
import os
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import numpy as np
#from plotly.offline import plotly as py
#from plotly.offline import graph_objs as go
#import plotly.plotly as py
#import plotly.graph_objs as go
#init_notebook_mode(connected=True)
from src.data import make_dataset
from dotenv import find_dotenv, load_dotenv
import os


%matplotlib inline
width = 10
height = 10
matplotlib.rcParams['figure.figsize'] = [width, height]
matplotlib.rcParams.update({'font.size': 14})
gt_lat_lng = (33.777351, -84.396541)
map_lat_lng = (33.775734, -84.381476)
# import matplotlib.style
# import matplotlib as mpl
# mpl.style.use('default')

(strava_data, drive_morning, drive_evening) = make_dataset.load_data()

# Access data store
#strava_data_h5 = pd.read_hdf(os.path.join('..', 'data', 'processed', 'strava_data.h5'))
#strava_data_h5 = pd.HDFStore(os.path.join('..', 'data', 'processed', 'strava_data.h5'))


# Retrieve data using key
#strava_data = strava_data_h5['strava_data']
#drive_morning = strava_data_h5['drive_morning']
#drive_evening = strava_data_h5['drive_evening']
#strava_data_h5.close()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
strava_data

,guid,external_id,upload_id,athlete,name,distance,moving_time,elapsed_time,total_elevation_gain,elev_high,...,partner_brand_tag,from_accepted_tag,segment_leaderboard_opt_out,highlighted_kudosers,laps,time,latlng,altitude,heartrate,temp
0,NaN,16389758747.tcx,1923757423,"{'firstname': None, 'lastname': None, 'profile...",Afternoon Ride,3.289973,00:17:54,00:18:11,60.2,304.0,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,"[90, 91, 91, 91, 91, 91, 91, 91, 92, 92, 92, 9...",NaN
1,NaN,16365415864.tcx,1923171624,"{'firstname': None, 'lastname': None, 'profile...",Morning Ride,2.988297,00:16:24,00:18:26,70.8,304.0,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,"[140, 140, 140, 140, 141, 142, 144, 145, 146, ...",NaN
2,NaN,16240774144.tcx,1907575620,"{'firstname': None, 'lastname': None, 'profile...",Afternoon Ride,3.069138,00:16:43,00:17:56,60.8,295.9,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,"[102, 102, 104, 105, 107, 108, 109, 111, 110, ...",NaN
3,NaN,09224f3d9599aa850193788e5608ae7c,1906538689,"{'firstname': None, 'lastname': None, 'profile...",Morning Ride,3.473091,00:18:30,00:20:13,70.0,296.0,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,16219678028.tcx,1906474740,"{'firstname': None, 'lastname': None, 'profile...",Morning Ride,3.383676,00:17:51,00:19:49,68.7,296.0,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,"[127, 127, 127, 127, 127, 127, 127, 127, 127, ...",NaN
5,NaN,16218810992.tcx,1905578192,"{'firstname': None, 'lastname': None, 'profile...",Evening Run,2.203879,00:20:41,00:20:49,36.0,304.6,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,"[140, 140, 140, 140, 140, 140, 140, 141, 141, ...",NaN
6,NaN,16193068760.tcx,1903069026,"{'firstname': None, 'lastname': None, 'profile...",Evening Ride,1.168240,00:06:32,00:15:47,23.2,288.5,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,"[99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 9...",NaN
7,NaN,22c722d7dad283b5bc5e6cd3a7d16644,1902719773,"{'firstname': None, 'lastname': None, 'profile...",Afternoon Ride,3.341174,00:15:35,00:17:14,65.5,296.0,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,16184546663.tcx,1902244152,"{'firstname': None, 'lastname': None, 'profile...",Afternoon Run,0.165844,00:02:14,00:29:46,0.0,279.5,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,"[50, 50, 50, 50, 75, 75, 75, 75, 69, 69, 69, 6...",NaN
9,NaN,39df1874faebd1a8afae9182991dc974,1901601125,"{'firstname': None, 'lastname': None, 'profile...",Morning Ride,3.580340,00:18:15,00:19:57,73.3,296.0,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def prepare_for_plotting(df):
    to_plot = df.copy()
    if 'moving_time' in to_plot.columns:
        to_plot['moving_time'] = to_plot.loc[:,'moving_time'] / pd.Timedelta(minutes=1)
    to_plot['elapsed_time'] = to_plot.loc[:,'elapsed_time'] / pd.Timedelta(minutes=1)
    return to_plot    

In [ ]:
def plot_polylines(df):
    m = folium.Map(map_lat_lng, zoom_start=14)
    for (index, row) in df.iterrows():
        if (row['map']['summary_polyline']):
            pl = polyline.decode(row['map']['summary_polyline'])
            bike_path = folium.PolyLine(pl).add_to(m)
    return m

In [ ]:
def get_start_latlng(mp):
    pl = mp['summary_polyline']
    if pl:
        return(polyline.decode(pl)[0])
    
def get_end_latlng(mp):
    pl = mp['summary_polyline']
    if pl:
        return(polyline.decode(pl)[-1])

In [ ]:
strava_data['start_latlng'] = strava_data['map'].apply(get_start_latlng)
strava_data['end_latlng'] = strava_data['map'].apply(get_end_latlng)
strava_data[['start_lat', 'start_lng']] = strava_data['start_latlng'].apply(pd.Series)
strava_data[['end_lat', 'end_lng']] = strava_data['end_latlng'].apply(pd.Series)

In [ ]:
morning_mask = strava_data['start_date_local'].dt.hour.isin(range(7, 11))
evening_mask = strava_data['start_date_local'].dt.hour.isin(range(14, 20))
weekday_mask = strava_data['start_date_local'].dt.weekday.isin(range(0, 5))
normal_time_mask = ((strava_data['moving_time'].dt.seconds / 60) > 10) & ((strava_data['moving_time'].dt.seconds / 60) < 30)
bike_mask = strava_data['type'] == 'Ride'
distance_mask = (strava_data['distance'] > 2.8) & (strava_data['distance'] < 4.7)
gtri_start_mask = strava_data['start_lat'] >= 33.781
gt_start_mask = (strava_data['start_lat'] <= 33.781) & (strava_data['start_lng'] <= -84.38)
gtri_end_mask = strava_data['end_lat'] >= 33.781
gt_end_mask = (strava_data['end_lat'] <= 33.781) & (strava_data['start_lng'] <= -84.38)

In [ ]:
def add_from_locations(df):
    df['from'] = ''
    df['from'] = df['from'].mask(evening_mask & weekday_mask & normal_time_mask & distance_mask & gt_start_mask, 'SC')
    df['from'] = df['from'].mask(evening_mask & weekday_mask & normal_time_mask & distance_mask & gtri_start_mask, 'GTRI')
    return df

def add_to_locations(df):
    df['to'] = ''
    df['to'] = df['to'].mask(morning_mask & weekday_mask & normal_time_mask & distance_mask & gt_end_mask, 'SC')
    df['to'] = df['to'].mask(morning_mask & weekday_mask & normal_time_mask & distance_mask & gtri_end_mask, 'GTRI')
    return df

In [ ]:
def heartrate_conversion(df):
    mask = ~pd.isnull(df['heartrate'])
    df.loc[mask, 'heartrate'] = (df[mask]['heartrate']).apply(ast.literal_eval)
    return df

In [ ]:
strava_data = heartrate_conversion(strava_data)

In [ ]:
morning_commutes = strava_data[morning_mask & weekday_mask & normal_time_mask & distance_mask & (gtri_end_mask | gt_end_mask)]
evening_commutes = strava_data[evening_mask & weekday_mask & normal_time_mask & distance_mask & (gtri_start_mask | gt_end_mask)]

In [ ]:
# Add columns to strava_data for the from and to strings for commutes
# This is duplication of information, but it makes it way easier for me to explore so it doesn't seem that horrible
strava_data = add_from_locations(strava_data)
strava_data = add_to_locations(strava_data)

In [ ]:
to_plot = prepare_for_plotting(morning_commutes)
plt.subplot(221)
plt.scatter(to_plot['start_date_local'].dt.to_pydatetime(), to_plot['moving_time'])
plt.subplot(222)
plt.scatter(to_plot['start_date_local'].dt.to_pydatetime(), to_plot['elapsed_time'])
plt.subplot(223)
plt.scatter(to_plot['start_date_local'].dt.to_pydatetime(), to_plot['moving_time'], color='C0')
plt.scatter(to_plot['start_date_local'].dt.to_pydatetime(), to_plot['elapsed_time'], color='C1')


plt.show()

In [ ]:
to_plot = prepare_for_plotting(morning_commutes)
to_plot[['moving_time', 'elapsed_time']].plot.box()
plt.show()


In [ ]:
to_plot = prepare_for_plotting(evening_commutes)
to_plot_car = prepare_for_plotting(drive_evening.dropna())
plt.subplot(221)
plt.scatter(to_plot['start_date_local'].dt.to_pydatetime(), to_plot['moving_time'])
plt.subplot(222)
plt.scatter(to_plot['start_date_local'].dt.to_pydatetime(), to_plot['elapsed_time'])
plt.subplot(223)
plt.scatter(to_plot['start_date_local'].dt.to_pydatetime(), to_plot['moving_time'], color='C0')
plt.scatter(to_plot['start_date_local'].dt.to_pydatetime(), to_plot['elapsed_time'], color='C1')
plt.scatter(to_plot_car['start_date_local'].dt.to_pydatetime(), to_plot_car['elapsed_time'], color='C2')
plt.legend(['Bicycle Moving Time', 'Bicycle Elapsed Time', 'Driving Elapsed Time'])

plt.show()

In [ ]:
# Creating a nice plot
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 10), sharey=True)

axes[0].scatter(to_plot['start_date_local'].dt.to_pydatetime(), to_plot['elapsed_time'], color='C0')
axes[0].scatter(to_plot_car['start_date_local'].dt.to_pydatetime(), to_plot_car['elapsed_time'], color='C1')
axes[0].legend(['Biking', 'Driving'], bbox_to_anchor=[.79,.97])
to_plot_bike = to_plot
to_plot['bike_moving_time'] = to_plot_bike['moving_time']
to_plot['Biking'] = to_plot_bike['elapsed_time']
to_plot['Driving'] = to_plot_car['elapsed_time']
to_plot[['Biking', 'Driving']].plot.box(ax=axes[1], showfliers=False)
fig.text(0.09, 0.5, 'Time (minutes)', ha='center', va='center', rotation='vertical')
axes[0].set_xlabel('Date')
fig.suptitle('Commuting by Bike vs. Car', fontsize = 20)
for ax in axes:
    ax.spines["top"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(False)
#plt.yticks(range(0, 91, 10), [str(x) + "%" for x in range(0, 91, 10)], fontsize=14)
#plt.xticks(fontsize=14)
for y in range(10, 36, 5):
    for ax in axes:
        ax.tick_params(axis="both", which="both", bottom=False, top=False,
        labelbottom=True, left=False, right=False, labelleft=True)
        ax.axhline(y=y, linestyle='--', lw=0.5, color='black', alpha=0.5)
        #ax.plot_date([to_plot_car['start_date_local'].min(), to_plot_bike['start_date_local'].max()], [y, y],  "--", lw=0.5, color="black", alpha=0.9)
axes[0].tick_params(axis='both', which="both", bottom=True)
axes[1].tick_params(axis='both', labelleft=False)
# plt.savefig("figures/to_post.png", bbox_inches="tight")
plt.show()


In [ ]:
to_plot_bike = prepare_for_plotting(evening_commutes)
to_plot_car = prepare_for_plotting(drive_evening)
to_plot = pd.DataFrame()
to_plot['bike_moving_time'] = to_plot_bike['moving_time']
to_plot['bike_elapsed_time'] = to_plot_bike['elapsed_time']
to_plot['car_elapsed_time'] = to_plot_car['elapsed_time']

to_plot[['bike_moving_time', 'bike_elapsed_time', 'car_elapsed_time']].plot.box()
plt.show()


In [ ]:
pl = polyline.decode(strava_data['map'][1]["summary_polyline"])
m = folium.Map(map_lat_lng, zoom_start=14)
bike_path = folium.PolyLine(pl).add_to(m)
m


In [ ]:
plot_polylines(evening_commutes)

In [ ]:
m = folium.Map(map_lat_lng, zoom_start=14)
end_latlng = []
for (index, row) in evening_commutes.iterrows():
    if (row['map']['summary_polyline']):
        pl = polyline.decode(row['map']['summary_polyline'])
        end_latlng.append(pl[1])
temp = list(zip(*end_latlng))
end_lat = temp[0]
end_lng = temp[1]
plt.scatter(end_lng, end_lat)
hline = [33.781 for i in range(0, len(end_lng))]
vline = [-84.38 for i in range(0, len(end_lat))]
plt.plot(end_lng, hline)
plt.plot(vline, end_lat)

In [ ]:
evening_commutes_gtri = strava_data[evening_mask & weekday_mask & normal_time_mask & distance_mask & gtri_start_mask]
evening_commutes_gt = strava_data[evening_mask & weekday_mask & normal_time_mask & distance_mask & gt_start_mask]

In [ ]:
temp= 'Index: ' + (pd.Series(to_plot.index)).apply(str)

In [ ]:
# Index printing issues: https://stackoverflow.com/questions/32801806/pandas-concat-ignore-index-doesnt-work
to_plot = prepare_for_plotting(evening_commutes)
#plt.scatter(to_plot['start_date_local'].dt.to_pydatetime(), to_plot['moving_time'])
#to_plot['index_str'] = 'Index: ' + pd.Series(to_plot.index).apply(str)
data = [
    go.Scatter(
        x = to_plot.start_date_local.dt.to_pydatetime(),
        y = to_plot.elapsed_time,
        hoverinfo = 'text',
        marker = dict(
            color = 'green'
        ),
        showlegend = False,
        mode = 'markers',
        text = to_plot.index
    )
]

layout = go.Layout(
    title = "Bike Evening Commutes",
    titlefont = dict(
        size = 10
    ),
    xaxis = dict(
        zeroline = True
    ),
    yaxis = dict(
    )
)



fig = {
    'data': [
        {
            'x': to_plot.start_date_local.dt.to_pydatetime(),
            'y': to_plot.elapsed_time,
            'mode': 'markers',
            'name': 'Biking',
            'text': to_plot.index
        }
    ]
}

fig2 = go.Figure(data=data, layout=layout)
# Plot and embed in ipython notebook!
iplot(fig2, filename='basic-scatter')

# or plot with: plot_url = py.plot(data, filename='basic-line')

In [ ]:
to_plot['index'].head()

In [ ]:
'Prepend' + to_plot['index']

In [ ]:
data = [
    go.Scatter(
        'x': to_plot.start_date_local.dt.to_pydatetime(),
        'y': to_plot.elapsed_time,
        hoverinfo = 'y',
        marker = dict(
            color = 'green'
        ),
        showlegend = False
    )
]

layout = go.Layout(
    title = "Set hover text formatting<br><a href= https://github.com/d3/d3-time-format/blob/master/README.md#locale_format>https://github.com/d3/d3-time-format/blob/master/README.md#locale_format</a>",
    titlefont = dict(
        size = 10
    ),
    xaxis = dict(
        zeroline = False
    ),
    yaxis = dict(
        hoverformat = '.2f'
    )
)

fig = go.Figure(data=data,layout=layout)
py.iplot(fig, filename = "format-hover-text")

In [ ]:
fig = {
    'data': [
  		{
  			'x': df2007.gdpPercap, 
        	'y': df2007.lifeExp, 
        	'text': df2007.country, 
        	'mode': 'markers', 
        	'name': '2007'},
        {
        	'x': df1952.gdpPercap, 
        	'y': df1952.lifeExp, 
        	'text': df1952.country, 
        	'mode': 'markers', 
        	'name': '1952'}
    ],
    'layout': {
        'xaxis': {'title': 'GDP per Capita', 'type': 'log'},
        'yaxis': {'title': "Life Expectancy"}
    }
}

In [ ]:
# https://markhneedham.com/blog/2017/04/29/leaflet-strava-polylines-osm/
# https://dzone.com/articles/strava-calculating-the-similarity-of-two-runs
# http://bl.ocks.org/matt-leach/5336dc8249cf4401fc0f6b31743202e5
# http://andykee.com/visualizing-strava-tracks-with-python.html
# https://gis.stackexchange.com/questions/42224/creating-polyline-based-heatmap-from-gps-tracks
# https://yizeng.me/2017/01/11/get-a-strava-api-access-token-with-write-permission/